# **Overall Standings & Performance Consistency Module**

Overall Standings module produces chart results for a particular season on the basis of #wins. Season is required as an input.

Performance Consistency module tracks a team's performance improvement consistency across all the seasons. Teams that have played for only one season are not considered.


In [56]:
import pyspark                          # for sparkContext and Dataframes
from pyspark.sql.types import *         # for defining schema with various datatypes
import pyspark.sql.functions as func    # for ETL, data processing on Dataframes

import pandas as pd                     # converting PysparkDF to PandasDF when passing it as a parameter to Bokeh invokes 

from datetime import *                  # for datetime datatype for schema
from dateutil.parser import parse       # for string parse to date

from bokeh.io import push_notebook, show, output_notebook  # various output methods for jupyter notebook
from bokeh.plotting import figure                          # creating a figure variable
from bokeh.charts import Bar, output_file, show            # creating bar charts, and displaying it
from bokeh.charts.attributes import cat                    # extracting column for 'label' category in bar charts
from bokeh.palettes import *                               # brewer color palette
from bokeh.models import Range1d                           # calibrating x/y-ranges of graph
output_notebook()

sql = SQLContext(sc)

Loading BokehJS ...

In [57]:
#Extracting and Transforming csv data

data_path = "../input/csv/"                                # path directory to input csv files
match_rdd = sc.textFile(data_path + "matches.csv")         # reading csv files into RDD

# Issue: Applying this custom schema generates error while calling various PysparkDF functions. e.g. show()
# header = match_rdd.first()
# fields = [StructField(field_name, StringType(), False) for field_name in header.split(',')]
# fields[0].dataType = LongType()
# fields[3].dataType = DateType()
# fields[9].dataType = BooleanType()
# fields[11].dataType = LongType()
# fields[12].dataType = LongType()
# fields[17].nullable = True
# schema = StructType(fields)

match_header = match_rdd.filter(lambda l: "id,season" in l)     # storing the header tuple
match_no_header = match_rdd.subtract(match_header)              # subtracting it from RDD
match_temp_rdd = match_no_header.map(lambda k: k.split(','))\
.map(lambda p: (int(p[0]), p[1],p[2],parse(p[3]).date(),p[4]\
                ,p[5],p[6],p[7],p[8],p[9]=='1',p[10],int(p[11])\
                ,int(p[12]),p[13],p[14],p[15],p[16],p[17]))     # Transforming csv file data

match_df = sql.createDataFrame(match_temp_rdd, match_rdd.first().split(','))  # converting to PysparkDF
match_df = match_df.orderBy(match_df.id.asc())                                # asc sort by id
match_df.show()

+---+------+----------+----------+--------------------+--------------------+--------------------+-------------+------+----------+--------------------+-----------+--------------+---------------+--------------------+-----------+--------------+--------------+
| id|season|      city|      date|               team1|               team2|         toss_winner|toss_decision|result|dl_applied|              winner|win_by_runs|win_by_wickets|player_of_match|               venue|    umpire1|       umpire2|       umpire3|
+---+------+----------+----------+--------------------+--------------------+--------------------+-------------+------+----------+--------------------+-----------+--------------+---------------+--------------------+-----------+--------------+--------------+
|  1|  2008| Bangalore|2008-04-18|Kolkata Knight Ri...|Royal Challengers...|Royal Challengers...|        field|normal|     false|Kolkata Knight Ri...|        140|             0|    BB McCullum|M Chinnaswamy Sta...|  Asad Rauf|   

In [58]:
# Overall ranking module

def overall_rank_func(season_num):
    overall_ranking = match_df.filter(match_df.season == season_num)\
                    .groupBy("winner").count().orderBy("count",ascending=0) # extracting required columns into another DF
    overall_ranking = overall_ranking.filter("winner != '' ")                          # Deleting records of tied matches
    overall_ranking = overall_ranking.selectExpr("winner as Teams", "count as Wins")   # Renaming columns
    overall_ranking.show(truncate=False)                                               

    overall_pdf = overall_ranking.toPandas()      # Converting to PandaDF
    N = overall_ranking.count()                   # Storing #ofTuples              
    clr=brewer["Spectral"][N]                     # Brewing color hex values for each tuple('team')

    p = Bar(overall_pdf, values="Wins", color="Teams",palette=clr,\
            label=cat(columns="Teams", sort=False), xgrid=True,\
            xlabel="Teams", ylabel="Wins", title="Overall Standings " + str(season_num),\
            legend='top_right', plot_width=950, bar_width=0.6)   # generating bar chart
    t = show(p, notebook_handle=True)                            # displaying the chart

overall_rank_func(2013)                                          # call this function by providing season year as arg

+---------------------------+----+
|Teams                      |Wins|
+---------------------------+----+
|Mumbai Indians             |13  |
|Chennai Super Kings        |12  |
|Rajasthan Royals           |11  |
|Sunrisers Hyderabad        |10  |
|Royal Challengers Bangalore|9   |
|Kings XI Punjab            |8   |
|Kolkata Knight Riders      |6   |
|Pune Warriors              |4   |
|Delhi Daredevils           |3   |
+---------------------------+----+



In [59]:
#Performance consistency module

def consistency_func():
    consistency_df = match_df.select("season","winner")\
    .groupBy("season","winner").count().orderBy("winner")                  # extracting required columns 
    
    consistency_df = consistency_df.filter("winner!='' ")                  # filtering out tied matches records

    # constraint : teams that haven't played more than one season aren't considered
    constraint_df = match_df.groupBy("winner","season")\
                .count().orderBy("winner")                                 # extracting list of season-wise winner teams
    constraint_df = constraint_df.groupBy("winner").count()\
                .filter("count > 1 and winner!='' ")                       # filtering out teams that don't satisfy constraint
    constraint_list = [i.winner for i in constraint_df.collect()]          # storing a list of filtered teams

    consistency_df = consistency_df.where(func.col("winner")\
                    .isin(constraint_list))                                # applying the constraint list
    consistency_df = consistency_df.groupBy("winner")\
                    .agg(func.stddev_pop("count").alias("stddev"),\
                    func.sum("count").alias("total_wins"))\
                    .orderBy("stddev","total_wins")                        # calculating the performance consistency
            
    consistency_df = consistency_df.withColumn("final_deviations",\
                    100-(consistency_df.stddev/consistency_df.total_wins)*100)\
                    .orderBy("final_deviations", ascending=False)          # scaling to appropriate scale
        
    consistency_df = consistency_df.selectExpr("winner as Teams", "final_deviations as Consistency")
                                                                           # renaming columns

    consistency_pdf = consistency_df.toPandas()                    # converting to PandasDF
    N = consistency_df.count()                                     # storing #oftuples('teams')
    clr=brewer["RdYlGn"][N]                                        # brewing colors hex values for each team

    p2 = Bar(consistency_pdf, values="Consistency",\
             color="Teams", palette=clr,\
             label=cat(columns="Teams", sort=False),\
             xlabel="Teams", ylabel="Win Consistency %age",\
             title="IPL Performance Consistencies",\
             legend='top_right', plot_width=950, bar_width=0.6)    # generating bar chart
    p2.y_range=Range1d(90,110)                                     # setting appropriate ranges
    t2 = show(p2, notebook_handle=True)                            # displaying chart

consistency_func()      # function to call consistency module